<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
from urllib.parse import quote_plus
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432
# P.S. на всякий случай не буду выкладывать данные в открытый доступ на GitHub

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select 
                count(*)
                from vacancies
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.iloc[0][0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''select
                count(*)
                from employers
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df.iloc[0][0]

23501

3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select
                count(*)
                from areas
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df.iloc[0][0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select
                count(*)
                from industries
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df.iloc[0][0]

294

In [12]:
# выводы по предварительному анализу данных

***

Видим, что на примерно 23 тысячи работодателей приходится около 50 тысяч ваканский, т.е. в среднем каждый работодатель выкладывает приблизительно две вакансии. 

Ещё можно отметить, что в вакансиях указано более тысячи различных регионов, что может помешать анализу данных и построению модели.

---

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
# Объединяем таблицы vacancies и areas по id и затем группируем по регионам. Выбираем название региона
# и количество вакансий (строк), относящихся к нему
query_4_1 = f'''select 
                    a.name area, count(*) cnt
                from vacancies v
                    join areas a on v.area_id = a.id
                group by a.id
                order by 2 desc
'''

In [14]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df.head()

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''select count(*)
                from vacancies
                where salary_from is not null or salary_to is not null
'''

In [16]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df.iloc[0][0]

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
# Из таблицы vacancies вычисляем и затем округляем до целого сначала среднюю минимальную зарплату 
# (столбец salary_from), а затем максимальную зарплату (столбец salary_to)
query_4_3 = f'''select
                    round(avg(salary_from)) avg_min_salary, round(avg(salary_to)) avg_max_salary
                from vacancies
'''

In [18]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,avg_min_salary,avg_max_salary
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
# В таблице vacancies группируем данные сначала по рабочему графику (schedule), а затем
# по типу трудоустройства (employment). Выбираем столбцы schedule, employment и считаем кол-во вакансий(строк)
# в каждой паре. Сортируем данные по числу вакансий в порядке убывания
query_4_4 = f'''select
                    schedule, employment, count(*)
                from vacancies 
                group by schedule, employment
                order by 3 desc
'''

In [20]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
# В таблице vacancies группируем данные по опыту (experience). Выбираем столбец experience и кол-во вакансий (строк)
# в каждой группе. Сортируем данные по числу вакансий в порядке возрастания
query_4_5 = f'''select 
                    experience, count(*)
                from vacancies 
                group by experience
                order by 2
'''

In [22]:
# результат запроса
exp = pd.read_sql_query(query_4_5, connection)
exp

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [23]:
# выводы по детальному анализу вакансий

***

1. Из первого запроса видим, что среди городов с наибольшим количеством вакансий присутствуют города из Беларуси (столица страны - Минск) и Казахстана (Алматы), что указывает на международный статус компании.
2. Второе запрос показывает, что только в половине вакансий указана хотя бы какая-то зарплата, что может помешать построению модели.
3. Из третьего запроса узнали среднюю зарплатную "вилку" - приблизительно от 70 - 110 тысяч рублей.
4. Четвертый запрос показал, что несмотря на мировую тенденцию перехода программистов на удаленную работу, по данным исследуемого сервиса такой тип занятости находится только на втором месте, сильно отставая от типа "полный день + полная занятость".
5. Из пятого запроса находим, что требуемый у большинства работодателей опыт работы - от 1 до 3 лет.

Таким образом, мы собрали много важной информации о вакансиях для соискателей работы.

---

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
# Объединяем таблицы vacancies и employers по столбцам employer_id и id соответственно. Группируем по
# id работодателя. Выбираем столбец имя работодателя и кол-во вакансий, которые предлагает каждый.
# Сортируем данные по числу вакансий в порядке убывания
query_5_1 = f'''select 
                    e.name, count(*)
                from vacancies v
                    join employers e on v.employer_id = e.id
                group by e.id
                order by 2 desc
'''

In [25]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df.head()

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
# К таблице areas с помощью left join присоединяем таблицы employers и vacancies по соответсвующим id.
# Далее сортируем данные так, чтобы в регионе не было вакансий. Группируем данные по id регионов, 
# сортируем данные по числу работодателей (employers_cnt) в порядке убывания и выбираем первую строку.
# Выбираем имя региона, кол-во работодателей в нём к кол-во вакансий
query_5_2 = f'''select 
                    a.name, count(e.id) employers_cnt, count(v.id) vacancies_cnt
                from areas a
                    left join employers e on e.area = a.id
                    left join vacancies v on v.area_id = a.id
                where v.id is null
                group by a.id
                order by 2 desc
                limit 1
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df.iloc[0][0]

'Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
# Соединяем таблицы employers и vacancies по соответсвующим id. Выбираем только те данные, в которых есть вакансии.
# Группируем данные по id работодателей, сортируем данные по числу регионов в порядке убывания и выбираем первого
# работодателя. Выбираем имя работодателя и кол-во регионов, в которых он выкладывает вакансии
query_5_3 = f'''select 
                    e.name, count(distinct v.area_id)
                from employers e
                    join vacancies v on v.employer_id = e.id
                where v.id is not null
                group by e.id
                order by 2 desc
                limit 1
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df.iloc[0][1]

181

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
# К таблице employers присоединяем с помощью left join таблицу employers_industries. 
# Оставляем только тех работодателей, у которых не указан id сферы деятельности. 
# Выбираем кол-во строк в получившейся таблице
query_5_4 = f'''select 
                    count(*)
                from employers e
                    left join employers_industries ei on e.id = ei.employer_id
                where employer_id is null
'''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df.iloc[0][0]

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
# К таблице employers присоединяем с помощью left join таблицу employers_industries. 
# Группируем по id работодателей и ставим условие: кол-во сфер деятельностей должно быть ровно 4.
# Сортируем названия компаний/имена работодателей в алфавитном порядке и выбираем третьего работодателя.
# Выбираем его имя
query_5_5 = f'''select 
                    e.name
                from employers e
                    left join employers_industries ei on e.id = ei.employer_id
                group by e.id
                having count(ei.industry_id) = 4
                order by 1
                offset 2 limit 1
'''

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df.iloc[0][0]

'2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
# Соединяем таблицы industries и employers_industries по соответствующим id.
# Оставляем только те вакансии, что связаны с разработкой программного обеспечения.
# Выбираем кол-во строк в полученной таблице
query_5_6 = f'''select 
                    count(*)
                from industries i
                    join employers_industries ei on i.id = ei.industry_id
                where i.name = 'Разработка программного обеспечения'
'''

In [35]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df.iloc[0][0]

3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-милионников
# С помощью метода quote_plus из библиотеки urllib.parse сохраняем переменную с ссылкой на сайт Википедии, 
# на котором есть таблица с названиями городов-миллиоников
url = quote_plus('https://ru.wikipedia.org/wiki/Города-миллионеры_России', safe=':/')
# Считываем данные с помощью библиотеки Pandas
df = pd.read_html(url)
# Выбираем первую таблицу и обращаемся к столбцу 'Город'. Преобразуем полученный объект Series в кортеж
mln_cities = tuple(df[0] ['Город'])

In [37]:
# текст запроса
# С помощью метода 'with ... as' присваиваем следующей таблице (*) имя temp.
# (*) Соединяем таблицы employers, vacancies и areas. Выбираем только те данные,
# в которых работодатель - 'Яндекс' и имя региона входит с список городов-миллиоников, полученных выше.
# Группируем данные по id регионов и сортируем по кол-ву вакансий.
# На выходе получаем таблицу со столбцами name (название города-миллионика) и cnt (кол-во вакансий в каждом из них)
# Таблицу temp объединяем с помощью union all c таблицей с одной строчкой и двумя столбцами- 'Total' и сумма 
# всех вакансий из таблицы temp
query_5_7 = f'''with temp as 
                (select 
                    a.name, count(v.id) cnt
                from employers e
                    join vacancies v on v.employer_id = e.id
                    join areas a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {mln_cities}
                group by a.id
                order by 2)
                select
                    temp.name, temp.cnt
                from temp
                union all
                select 
                    'Total', sum(temp.cnt)
                from temp
'''

In [38]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


In [39]:
# выводы по анализу работодателей

***

1. Из первого запроса узнали топ-5 компаний по кличеству вакансий. Как видим, больше всего вакансий (почти 2000) у Яндекса, на втором месте Ростелеком, который предлагает в примерно 4 раза меньше мест.
2. Четвертый запрос показал, что у 8.5 тысяч работодателей не указана сфера деятельности, что мешает соискателям понять, подходят ли они на вакансию такого работодателя, а нам мешает построить модель машинного обучения.
3. Из последнего запроса получили, что самый крупный работодатель (Яндекс) предлагает только четверть вакансий в городах-миллиониках. Возможно, компания старается больше развиваться в регионах или отдельных странах, но, скорее всего, компания в большинстве случаев предлагает удалённую работу, поэтому ей не важно место проживания соискателя.

---

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query_6_1 = f'''select 
                    count(*)
                from vacancies
                where name ilike '%data%' or name ilike '%данн%'
'''

In [41]:
# результат запроса
res = pd.read_sql_query(query_6_1, connection)
res.iloc[0][0]

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [42]:
# текст запроса
# Из таблицы vacancies выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста, и одновременно есть указание на уровень специалиста Junior.
# Выбираем число вакансий, подходящих под данные условия
query_6_2 = f'''select
                    count(id)
                from vacancies
                where (name ilike '%data science%' or name ilike '%data scientist%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%') or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    and (name ilike '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка')
'''

In [43]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df.iloc[0][0]

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [44]:
# текст запроса
# Из таблицы vacancies выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста, и одновременно есть указание на требование знания языка PostgreSQL.
# Выбираем число вакансий, подходящих под данные условия
query_6_3 = f'''select
                    count(id)
                from vacancies
                where (name ilike '%data science%' or name ilike '%data scientist%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%ml%' and name not like '%HTML%' and name not like '%QML%'   
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    and (key_skills like '%SQL%' or key_skills like '%postgres%')
'''

In [45]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df.iloc[0][0]

229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [46]:
# текст запроса
# Из таблицы vacancies выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста, и одновременно есть указание на требование знания языка Python.
# Выбираем число вакансий, подходящих под данные условия
query_6_4 = f'''select
                    count(id)
                from vacancies
                where (name ilike '%data science%' or name ilike '%data scientist%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%ml%' and name not like '%HTML%' and name not like '%QML%'   
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    and key_skills ilike '%python%'
'''

In [47]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df.iloc[0][0]

357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [48]:
# текст запроса
# Из таблицы vacancies выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста, и одновременно есть указание на хотя бы какие-то навыки.
# Вычисляем среднее кол-во востребованных навыков, вычитая длину полной строки и той, в которой есть табуляция
# (т.е. указание на новый навык). Округляем результат до двух знаков после запятой
query_6_5 = f'''select
                    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2)
                from vacancies
                where
                    (name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name like '%ML%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    and name not ilike '%HTML%'
                    and key_skills is not null
'''

In [49]:
# результат запроса
pd.set_option('display.max_rows', None)
df = pd.read_sql_query(query_6_5, connection)
df.iloc[0][0]

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [50]:
# текст запроса
# Из таблицы vacancies выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста. Группируем данные по опыту работу и сортируем в порядке убывания по
# средней зарплате. 
# Выбираем группы опыта работы и столбец avg_salary, в котором указана средняя зарплата по группе
query_6_6 = f'''select
                    experience, 
                    round(avg(coalesce((salary_to+salary_from)/2, salary_from, salary_to)), 0) avg_salary
                from vacancies
                where (salary_from is not null or salary_to is not null)
                    and (name ilike '%data science%' or name ilike '%data scientist%'
                                    or name ilike '%исследователь данных%'
                                    or name like '%ML%' and name not ilike '%HTML%'
                                    or name ilike '%machine learning%'
                                    or name ilike '%машинн%обучен%')
                group by experience
                order by 2 desc
'''

In [51]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,avg_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


In [52]:
# выводы по предметному анализу

***

1. Видим, что среди почти 50 тысяч вакансий только 1771 связаны с Дата-аналитикой, и только 51 вакансия подходит начинающим специалистам.
2. Только в 229 вакансиях для дата-сайентистов требуют знания языка SQL или умение работать с базой данных с помощью Postrgre.
3. Знания языка Python требуют в менее чем 20 процентах вакансий.
4. В среднем соискателю нужно знать 6-7 ключевых навыков в сфере Data Science, чтобы получить нужную должность.
5. Из последнего запроса узнаём среднюю зарплату дата-сайентиста в зависимости от опыта работы: даже для соискателей без опыта средняя зарплата составляет около 75 тысяч рублей. Для опытных работников зарплата может достигать 250 тысяч и выше.

---

# Общий вывод по проекту

In [53]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

---

Перед тем, как сделать общий вывод, хотелось бы уделить чуть больше внимания предметному анализу для профессии дата-сайентиста и практически не использовавшейся таблице industries. Я поставил задачу: найти количество вакансий и среднюю зарплату по каждой сфере.

In [54]:
# запрос
# Объединяем таблицы industries, employers_industries, employers и vacancies по соответствующим id.
# Выбираем только те вакансии, в которых есть упоминание одного из ключевых слов,
# указывающих на профессию дата-сайентиста. Группируем данные по id сфер, сортируем сначала по кол-ву вакансий
# в порядке убывания, а затем по средней зарплате в том же порядке.
# Выбираем имя сферы, кол-во вакансий и среднюю зарплату
query_bonus = f'''select
                        i.name, 
                        count(v.id) num_of_vacancies, 
                        round(avg(coalesce((v.salary_to+v.salary_from)/2, v.salary_from, v.salary_to)), 0) avg_salary
                    from industries i
                        join employers_industries ei on i.id = ei.industry_id
                        join employers e on e.id = employer_id
                        join vacancies v on v.employer_id = e.id
                    where (v.name ilike '%data science%' or v.name ilike '%data scientist%'
                                                        or v.name ilike '%исследователь данных%'
                                                        or v.name like '%ML%' and v.name not ilike '%HTML%'
                                                        or v.name ilike '%machine learning%'
                                                        or v.name ilike '%машинн%обучен%')
                    group by i.id
                    order by 2 desc, 3 desc nulls last
'''

In [55]:
result = pd.read_sql_query(query_bonus, connection)
result

,name,num_of_vacancies,avg_salary
0,Разработка программного обеспечения,148,212308.0
1,"Системная интеграция, автоматизации технологи...",139,212150.0
2,Банк,92,37000.0
3,"Интернет-компания (поисковики, платежные систе...",79,178739.0
4,Мобильная связь,23,NaN
5,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22,350000.0
6,"Управляющая, инвестиционная компания (управлен...",21,200000.0
7,Консалтинговые услуги,16,222500.0
8,Фиксированная связь,13,NaN
9,"Научно-исследовательская, научная, академическ...",9,115000.0


Получилась достаточно информативная таблица, из которой узнаём:
1. Насколько разнообразны вакансии для дата сайентистов: от разработки программного обеспечения до энергетики, от сферы финансов до юриспруденции. Видно, насколько обширна данная профессия.
2. Количество вакансий по каждой сфере: ожидаемо, больше всего вакансий связаны с программированием и информационными технологиями, однако специалисты Data Science востребованы и в банках, коммуникационных, маркетинговых компаниях и т.д.
3. Среднюю зарплату, хоть не во всех, но зато в тех сферах, где больше вакансий. Например, видим, что самые высокие зарплаты предлагают в сфере маркетинга (350 тысяч) и лизинговых (300 тысяч) компаниях, причем сфера маркетинга еще и входит в топ-6 по количеству вакансий.

---

Также для вывода будет полезно рассмотреть общее распределение вакансий и зарплат по сферам.

In [56]:
# Объединяем таблицы industries, employers_industries, employers и vacancies по соответствующим id.
# Группируем данные по id сфер, сортируем сначала по кол-ву вакансий в порядке убывания, а затем 
# по средней зарплате в том же порядке.
# Выбираем имя сферы, кол-во вакансий и среднюю зарплату
query_bonus_1 = f'''select
                        i.name, 
                        count(v.id) num_of_vacancies, 
                        round(avg(coalesce((v.salary_to+v.salary_from)/2, v.salary_from, v.salary_to)), 0) avg_salary
                    from industries i
                        join employers_industries ei on i.id = ei.industry_id
                        join employers e on e.id = employer_id
                        join vacancies v on v.employer_id = e.id
                    group by i.id
                    order by 2 desc, 3 desc nulls last
'''

In [57]:
res = pd.read_sql_query(query_bonus_1, connection)
res.head(10)

,name,num_of_vacancies,avg_salary
0,Разработка программного обеспечения,12499,117659.0
1,"Системная интеграция, автоматизации технологи...",11034,102285.0
2,"Интернет-компания (поисковики, платежные систе...",6413,70462.0
3,Банк,2742,84009.0
4,Мобильная связь,1585,52300.0
5,Кадровые агентства,1143,144530.0
6,Консалтинговые услуги,1137,91281.0
7,Интернет-провайдер,1095,59111.0
8,Розничная сеть (продуктовая),1034,69396.0
9,Фиксированная связь,1028,49626.0


Видим, что с большим отрывом от остальных лидируют профессии, связанные с программированием. Найдём процент вакансий, которые занимают первые три сферы.

In [58]:
percent = round(100*(res.iloc[0]['num_of_vacancies'] + res.iloc[1]['num_of_vacancies'] \
    + res.iloc[2]['num_of_vacancies'])/sum(res['num_of_vacancies']))
print(f'Вакансии, связанные с программированием, составляют {percent}% от общего количества')

Вакансии, связанные с программированием, составляют 37% от общего количества


---

Вдобавок, для общего вывода пригодится наглядная диаграмма опыта работы, полученная из задания 4.5

In [59]:
# Построим круговую диаграмму с помощью библиотеки Plotly
fig = px.pie(
    exp,
    names = 'experience',
    values = 'count',
    title = 'Распределение опыта работы'
)

fig.show()

In [60]:
connection.close()

---

# Вывод

Был проведён тщательный анализ вакансий, размещенными работодателями, в частности:
<br>
<br>

* Общий анализ, в ходе которого получена информация о масштабе предоставленных данных: так, главная таблица vacancies состоит из почти 50 тысяч строк.

* Детальный анализ вакансий, с помощью которого были получены данные, полезные для соискателей.

* Анализ работодателей, который также важен для соискателей, а также для построения модели машинного обучения.

* Предметный анализ, с помощью которого были изучены вакансии, связанные с данными. 

<br>
<br>

*Если в общем и целом говорить про вакансии, то большинство предлагаемых рабочих мест связано с программированием. Больше всего работодатели предлагают работу в крупных городах России и СНГ, среди которых лидируют Москва и Санкт-Петербург, что ожидаемо, т.к. у крупных российских компаний есть тенденция размещать офисы именно в этих двух городах. Работу предлагают преимущественно в формате 'полный рабочий день + полная занятость', на втором месте, с сильно меньшей частотой - 'удалённая работа + полная занятость'. Самое большое количество вакансий предлагает компания Яндекс, что можно было ожидать, исходя из того, что на данный момент это самый крупный и быстроразвивающийся IT-бизнес в России. Средняя заработная плата - от 70 до 110 тысяч рублей, что выше средней по Росси; однако, учитывая, что в предоставленном датафрейме больше половины работодетелей (53%) требуют опыт работы от 1 года до 3 лет и еще 14% не требуют опыта работы вообще, а также то, что около 40% вакансий связаны с программированием, что, очевидно, означает более высокие рсдение зарплаты, данные вполне сходятся с другими источниками.*
<br>
<br>

**На выходе мы получили и проанализировали огромное количество данных, полученных из базы данных, с помощью которых можно построить алгоритм, рекомендующий самые подходящие вакансии соискателям.**